# Generative Adversarial Networks

* Un GAN es como una batalla entre dos adversarios
    * Un generador
    * Un discriminador
* El generador intenta convertir ruido aleatorio en observaciones que parecen como si han sido muestreadas del dataset original
* El discriminador intenta predecir si una observacion viene del dataset original o si es una creada por el generador

## IMAGEN

* Al inicio del proceso el generador va a crear imagenes ruidosas y las predicciones del discriminador van a ser aleatorias
* La clave de los GANs yace en como alternar el entrenamiento de las dos redes:
    * el generador debe volverse mejor en enganiar al discriminador
    * el discriminador debe adaptarse para mantener su habilidad de identificar que observaciones son falsas
    * Esto causa que el generador encuentra nuevas formas de enganiar al discriminador y el ciclo continua

## El Discriminador

* La meta del discriminador es predecir si una imagen es real o falsa
    * Esto es un problema de clasificacion de imagenes supervisada.
    * Por esta razon podemos usar la misma arquitectura de red que vimos la clase anterior: un stack de convolutional layers seguidas por un fully connected output layer
* En el paper original del GAN, se usaron fully connected layers en vez de convolutional layers.
    * Sin embargo, desde entonces, se ha comprobado que las convolutional layers proveen mejor rendimiento para el discriminador
* Puede ser que vean un tipo de GAN llamado DCGAN (deep convolutional generative adversarial network) en la literatura.
    * Ahora todas las arquitecturas de GANs contienen convolutional layers, entonces el DC esta implicito.
* Tambien es comun ver batch normalization layers en el discriminador.

### Arquitectura

1. Definir el input al discriminador (la imagen)
2. Apilar convolutional layers
3. Aplanar la ultima convolutional layer a un vector
4. Capa lineal de una unidad, con una funcion de activacion sigmoide que transforma el output a un rango [0,1]

## El Generador

* El input al generador es un vector, usualmente muestreado de una distribucion normal multivariable.
* El output es una imagen del mismo tamanio que una imagen en el training set original.
* Esta descripcion es parecida al decoder en el VAE.
    * De hecho, el generador de un GAN cumple el mismo proposito que el decoder de un VAE
    * Convertir un vector en el espacio latente -> imagen
* El concepto de mapear de un espacio latente de regreso a al dominio original es muy comun en modelado generativo.
    * Nos provee la habilidad de manipular vectores en el espacio latente para cambiar high-level features de imagenes en el dominio original.

### Upsampling

* En el decoder del VAE que vimos, duplicamos el ancho y la altura del tensor en cada layer usando `ConvTranspose2d` layers.
    * Esto insertaba ceros entre pixeles antes de realizar las operaciones del convolution
* El objetivo de esto es transformar el input de regreso al dominio de la imagen original.
* Como vimos en el VAE los `ConvTranspose2d` pueden generar artefactos
    * pequenios patrones de ajedrez en las imagenes de output
    * eso tiene un impacto en la calidad de la imagen generada

## IMAGEN

## Entrenamiento del GAN

* Como hemos visto, la arquitectura del generado y el discriminador en un GAN es bastante simple
* La clave es entender el proceso de entrenamiento

* Podemos entrenar el discriminador creando un training set donde algunas imagenes son seleccionadas de forma aleatoria
    * observaciones _reales_ del training set
    * outputs del generador
    * La respuesta seria 1 para imagenes reales y 0 para imagenes generadas

* Entrenar el generador es mas dificil:
    * No tenemos un training set que nos indique la imagen _verdadera_ a la que un punto en particular en el espacio latente debe ser mapeada
    * En vez, lo que queremos es que la imagen generada enganie al discriminador
    * Cuando la imagen ingrese al discriminador el output deberia ser cercano a 1

* Por tanto, para entrenar el generador, primero necesitamos conectarlo al discriminador para crear un modelo que podamos entrenar.
* Especificamente, le alimentamos el output del generador al discriminador
    * el output de este modelo combinado es la probabilidad de que la imagen generada sea _real_ de acuerdo al discriminador
* Podemos entrenar este modelo combinado creando batches de entrenamiento que consisten de:
    * vectores de 100-dimensiones generadas de forma aleatoria
    * un label 1, ya que queremos entrenar al generador a producir imagenes que el discriminador piensa que son reales.

* El loss function entonces es solo el binary cross-entropy loss entre el output del discriminador y el vector de labels de 1s.

* Es importante que congelemos los pesos del discriminador mientras estamos entrenando el modelo combinado
    * De esta forma solo los pesos del generador se actualizan.
* Si no congelamos los pesos del discriminador, el discriminador se va a ajustar para que sea mas probable que prediga que las imagenes generadas son reales. Esto no es lo que queremos.
* Queremos que las imagenes generadas sean predecidas para tener un valor cercano a 1 (real) porque el generador es fuerte y no porque el discriminador es debil

## Desafios del GAN

* Aunque los GANs fueron un gran avance para el modelado generativo tambien son notoriamente dificiles de entrenar.
* A continuacion vamos a ver los problemas mas comunes
* Luego algunos ajustes al framework de un GAN para remediar los problemas

### Oscillating Loss

* El loss del discriminador y un generador empiezan oscilar bastante, en vez de exhibir estabilidad a largo plazo.
* Es normal que hayan oscilaciones entre batches, pero a largo plazo deberiamos buscar un loss que se estabiliza a largo plazo.

### Mode Collapse

* Ocurre cuando el generador encuentra pequenios numeros de muestras que enganian al discriminador
    * deja de producir ejemplos afuera de este conjunto limitado

### Uninformative Loss

* Dado que el modelo es creado para minimizar el loss function, es natural pensar que mientras mas pequenia sea el loss del generador, mejor la calidad de imagenes producidas
* Sin embargo, como el generador solo es calificado contra el discriminador actual y el discriminador esta constantemente mejorando, no podemos comparar el loss function evaluado en diferentes puntos a lo largo del proceso de entrenamiento.
* El loss del generador puede estar incrementando a traves del tiempo, aunque la calidad de imagenes mejore.
* Esta falta de correlacion entre el loss del generador y la calidad de imagenes hace dificil monitorear el entrenamiento.

## Soluciones para los problemas del GAN

* En anios recientes, avances clave han mejorado drasticamente la estabilidad de los GANs y disminuido la frequencia de los problemas que vimos.
* Vamos a explorar dos de estos avances
    * Wasserstein GAN (WGAN)
    * Wassertein GAN-Gradient Penalty (WGAN-GP)

### Wasserstein GAN

* Uno de los primeros grandes avances para estabilizar el entrenamiento de GANs.
* Con algunos cambios, los autores demostraron como entrenar GANs que tienen las siguientes propiedades:
    * Una metrica de loss con significado que se correlaciona con la convergencia del generador y la calidad de las muestras.
    * Mejora en la estabilidad del proceso de optimizacion
    
* Especificamente este paper introduce una nueva loss function para el discriminador y el generador.
* Usando esta loss function en vez de binarry cross entropy resulta en una convergencia mas estable del GAN.

### Wasserstein Loss

$$ -\frac{1}{n}\sum_{i=1}^{n}(y_i p_i) $$

In [ ]:
def wasserstein(y_true, y_pred):
    return -np.mean(y_true * y_pred)

### Lipschitz Constraint

* Como el wasserstein loss ya no restringe los outputs al rango $[0,1]$ si no a $[-\infty, \infty]$, el loss puede ser bastante grande.
* Los autores del loss demostraron que para que el Wasserstein loss funcione, hay que agregar restricciones adicionales.
* Especificamente se requiere que el critico (discriminador) sea _1-Lipschitz continuous function_. 

* El critic es una funcion D que convierte una imagen en una prediccion.
* Decimos que esta funcion es 1-Lipschitz si satisface la siguiente desigualdad para dos imagenes input, $x_1$ y $x_2$:

$$ \frac{\lvert D(x_1) - D(x_2)\rvert}{\lvert x_1 - x_2 \rvert} $$

* Aqui, el denominador es la diferencia absoluta promedio entre dos imagenes (a nivel pixel)
* El numerador es la diferencia absoluta entre las predicciones del critic.
* En esencia, requerimos un limite sobre la razon de cambio en la que las predicciones del critic pueden cambiar entre dos imagenes.

### Weight Clipping

* En el WGAN paper, los autores demuestran que es posible hacer cumplir el Lipschitz constraint haciendo clipping de los pesos del critic para que se encuentren en un rango pequenio, $[-0.01, 0.01]$ despues de cada batch en entrenamiento.